# CSV a txts

Vamos a generar los inserts para las tablas de mi proyecto de fifa db.

In [49]:
x = 1

Contador:

In [50]:
x = x + 1
x

2

Importar librerías

In [51]:
import pandas as pd
import numpy as np

Abrir CSV

In [52]:
ds_ASP = pd.read_csv('ASP.csv', sep=',')

Ejemplos

In [53]:
ds_ASP['Cliente']

0                (42206340) DANIELLA MARIA BOLAÑOS GAMERO
1                         (20100115663) PANDERO S.A. EAFC
2       (09468059) KATIA NATHALI DE LOAYZA WONG DE PAC...
3        (46472213) JHONATHAN MITCHELL ANTEZANA ESCALANTE
4                (42607724) KRISCIA ZULAY REATEGUI ZAMORA
                              ...                        
1253                                                  NaN
1254                                                  NaN
1255                                                  NaN
1256                                                  NaN
1257                                                  NaN
Name: Cliente, Length: 1258, dtype: object

<p>Filtrado de colores</p>

In [54]:
colors_rep = ds_ASP['Color']
colors = []

for c in colors_rep:
    if isinstance(c, str) and c.strip() != '' and c.lower() != 'nan':
        if c not in colors:
            colors.append(c)

colors = sorted(colors)

text_colors = '-- migrate:up\n\n'
id = 1

for c in colors:
    c_escaped = c.replace("'", "''")
    text_colors += f"INSERT INTO colors (id, nombre) VALUES ({id}, '{c_escaped}');\n"
    id += 1

text_colors += '\n-- migrate:down\nDELETE FROM colors;'
with open('inserts_colors.sql', 'w') as f:
    f.write(text_colors)


<h3>Descomposición en marca (nombre) y modelo (nombre, version, año)</h3>

In [55]:
import re

# 1. Columna del CSV
modelos_rep = ds_ASP['Modelo']
modelos = [] # Lista donde se guardarán los modelos limpios (sin duplicados)
marcas = [] # Lista donde se guardarán las marcas

# 2. Lista donde se guardarán los modelos limpios (sin duplicados)
modelos = []

# 3. Versiones y marcas válidas (catálogo controlado)
VERSIONES_VALIDAS = [
    'TITANIUM', 'TREND', 'XLS', 'XLT', 'XL', 'LTD',
    'PLATINUM', 'RAPTOR', 'BADLANDS', 'LARIAT',
    'ACTIVE', 'ST', 'PREMIUM', 'PRO', 'MAX',
    'BIG BEND', 'TREMOR', 'GT'
]

MARCAS_VALIDAS = [
    'TERRITORY', 'RANGER', 'EXPLORER', 'ESCAPE',
    'F-150', 'BRONCO', 'MAVERICK', 'MUSTANG',
    'EXPEDITION', 'TIGGO', 'ARRIZO', 'NEW TIGGO'
]

# 4. Función de filtrado y extracción
def filtrar_modelo(texto):
    if not isinstance(texto, str):
        return None

    texto = texto.strip()

    # Año: últimos 4 dígitos
    match_año = re.search(r'(20\d{2})$', texto)
    if not match_año:
        return None
    anio = match_año.group(1)

    # Versión
    version_modelo = None
    for v in VERSIONES_VALIDAS:
        if f' {v} ' in f' {texto} ':
            version_modelo = v
            break
    if version_modelo is None:
        return None

    # Marca
    marca = None
    for m in MARCAS_VALIDAS:
        if texto.startswith(m):
            marca = m
            break
    if marca is None:
        return None

    return {
        'version_modelo': version_modelo,
        'anio': anio,
        'marca': marca
    }

# 5. Procesamiento de los modelos
for fila in modelos_rep:
    data = filtrar_modelo(fila)

    if data:
        # Marcas únicas
        if data['marca'] not in marcas:
            marcas.append(data['marca'])

        # Modelos únicos (versión + año + marca)
        modelo_key = (data['version_modelo'], data['anio'], data['marca'])
        if modelo_key not in modelos:
            modelos.append(modelo_key)

# 6. Generación de los inserts SQL para MARCAS
sql_marcas = '-- migrate:up\n\n'
marca_ids = {}
id_marca = 1

for m in sorted(marcas):
    sql_marcas += f"INSERT INTO marcas (id_marca, nombre) VALUES ({id_marca}, '{m}');\n"
    marca_ids[m] = id_marca
    id_marca += 1

sql_marcas += '\n-- migrate:down\nDELETE FROM marcas;\n'

with open('inserts_marcas.sql', 'w') as f:
    f.write(sql_marcas)

# 7. Generación de los inserts SQL para MODELOS
sql_modelos = '-- migrate:up\n\n'
id_modelo = 1

for version_modelo, anio, marca in modelos:
    marca_id = marca_ids[marca]
    sql_modelos += (
        f"INSERT INTO modelos (id_modelo, version_modelo, anio, marca_id) "
        f"VALUES ({id_modelo}, '{version_modelo}', '{anio}-01-01', {marca_id});\n"
    )
    id_modelo += 1

sql_modelos += '\n-- migrate:down\nDELETE FROM modelos;\n'

with open('inserts_modelos.sql', 'w') as f:
    f.write(sql_modelos)


<p>Filtrado de GPS</p>
